MAKEHF1DATA

In [7]:
import openpyxl


In [57]:
import pandas as pd
import statsmodels.api as sm

# Load the merged data
merged_df = (pd.read_csv('Partial.csv', sep=';', low_memory=False))
print(merged_df.head())
# Load the mydate converter data
mydate_converter = pd.read_csv('mydate_converter.csv', names=['year', 'month', 'mydate'], skiprows=[0])


# Convert 'date' to datetime format and extract year and month
merged_df['date'] = pd.to_datetime(merged_df['date'], format='%d/%m/%Y', errors='coerce')
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month

# Ensure the 'aum' and 'ret' columns are properly formatted
merged_df['aum'] = merged_df['aum'].astype(str).str.replace(',', '.').astype(float)
merged_df['ret'] = merged_df['ret'].astype(str).str.replace(',', '.').astype(float)
# Merge with mydate_converter
mydate_converter = mydate_converter[pd.to_numeric(mydate_converter['year'], errors='coerce').notnull()]
mydate_converter['year'] = mydate_converter['year'].astype(int)
mydate_converter['month'] = mydate_converter['month'].astype(int)
mydate_converter['mydate'] = mydate_converter['mydate'].astype(int)

# Merge with mydate_converter
merged_df = merged_df.merge(mydate_converter, on=['year', 'month'], how='left')

# Step 2: Filter and Clean the Data
# Keep records from 1994 onwards
merged_df = merged_df[merged_df['year'] >= 1994]

# Generate elapsed time
merged_df['maxmydate'] = pd.to_numeric(merged_df.groupby('id')['mydate'].transform('max'), errors='coerce')
merged_df['minmydate'] = pd.to_numeric(merged_df.groupby('id')['mydate'].transform('min'), errors='coerce')
merged_df['elapsedtime'] = merged_df['maxmydate'] - merged_df['minmydate'] + 1

# Filter out invalid records
merged_df = merged_df[(merged_df['aum'] <= 100000000000) & (merged_df['aum'] >= 1000000)]
merged_df = merged_df[merged_df['ret'] <= 1000]

# Identify sporadic reporters
retcounter = merged_df.groupby('id').size().reset_index(name='ret_counter')

merged_df = merged_df.merge(retcounter, on='id', how='left')
merged_df['sporadic_dum'] = (merged_df['elapsedtime'] > merged_df['ret_counter']).astype(int)
max_sporadic = merged_df.groupby('id')['sporadic_dum'].transform('max')
merged_df = merged_df[max_sporadic == 0]
# Drop funds with fewer than 12 months of data
merged_df = merged_df[merged_df['ret_counter'] >= 12]
merged_df = merged_df[merged_df['ret'].notna()]

# Drop unnecessary columns
merged_df.drop(columns=['elapsedtime', 'ret_counter'], inplace=True)

# Save the cleaned data
merged_df.to_csv('tass2.csv', index=False)

# Step 3: Break Data into Pre-Crisis, Crisis, and Post-Crisis Periods
# Split data into pre-crisis, crisis, and post-crisis periods
tass_pre = merged_df[merged_df['mydate'] < 574]
tass_pre.to_csv('tass2_pre.csv', index=False)

tass_crisis = merged_df[(merged_df['mydate'] >= 574) & (merged_df['mydate'] <= 593)]
tass_crisis.to_csv('tass2_crisis.csv', index=False)

tass_post = merged_df[merged_df['mydate'] > 593]
tass_post.to_csv('tass2_post.csv', index=False)

# Step 4: Perform AR1 Adjustment
def ar1_adjustment(df, period_name):
    df = df.sort_values(by=['id', 'mydate'])
    df['rho'] = 0.0
    unique_ids = df['id'].unique()

    for unique_id in unique_ids:
        sub_df = df[df['id'] == unique_id]
        if len(sub_df) > 1:
            model = sm.OLS(sub_df['ret'].iloc[1:], sm.add_constant(sub_df['ret'].shift(1).iloc[1:])).fit()
            rho = model.params.iloc[1] if len(model.params) > 1 else 0  # Use iloc for positional access
            df.loc[df['id'] == unique_id, 'rho'] = rho

    df['ret_star'] = (df['ret'] - df['rho'] * df['ret'].shift(1)) / (1 - df['rho'])
    df.to_csv(f'tass4_{period_name}.csv', index=False)
    return df

# AR1 adjustment for pre-crisis period
tass_pre_adjusted = ar1_adjustment(tass_pre, 'pre')

# AR1 adjustment for crisis period
tass_crisis_adjusted = ar1_adjustment(tass_crisis, 'crisis')

# AR1 adjustment for post-crisis period
tass_post_adjusted = ar1_adjustment(tass_post, 'post')



    id        date       ret        aum  companyid         Main Strategy  \
0  862  31/01/2002  0,008599  975684231         66  European Long Biased   
1  862  31/03/2003  0,011499  873171102         66  European Long Biased   
2  862  30/06/2003  0,018399  746943753         66  European Long Biased   
3  862  31/05/2003  0,014799  742376533         66  European Long Biased   
4  862  29/02/2004  0,018899  822674719         66  European Long Biased   

  incentivefee managementfee  
0          0,2          0,01  
1          0,2          0,01  
2          0,2          0,01  
3          0,2          0,01  
4          0,2          0,01  


In [58]:
import pandas as pd
import statsmodels.api as sm

def load_and_clean_data_with_mydate(file_path, mydate_converter, keep_columns=None, drop_na_columns=None):
    file_ext = file_path.split('.')[-1]
    if file_ext == 'csv':
        df = pd.read_csv(file_path)
    elif file_ext in ['xls', 'xlsx']:
        df = pd.read_excel(file_path)

    # Merge with mydate_converter
    mydate_converter = mydate_converter[pd.to_numeric(mydate_converter['year'], errors='coerce').notnull()]

    mydate_converter['year'] = mydate_converter['year'].astype(int)
    mydate_converter['month'] = mydate_converter['month'].astype(int)
    mydate_converter['mydate'] = mydate_converter['mydate'].astype(int)

    df = df.merge(mydate_converter, on=['year', 'month'], how='left')
    
    if drop_na_columns:
        df = df.dropna(subset=drop_na_columns)
    if keep_columns:
        df = df[keep_columns]
    
    df = df.sort_values(by=['year', 'month'])
    return df

# Load the mydate converter data
mydate_converter = pd.read_csv('mydate_converter.csv', names=['year', 'month', 'mydate'], skiprows=[0])

# Load factor files with mydate
df_ff = load_and_clean_data_with_mydate('Factors/Corrected_FF_Research_Data_Factors.csv', mydate_converter, drop_na_columns=['month'])
df_fung_hsieh = load_and_clean_data_with_mydate('Factors/TF-Fac.xlsx', mydate_converter, keep_columns=['PTFSBD', 'PTFSFX', 'PTFSCOM', 'year', 'month'], drop_na_columns=['year'])
df_mom = load_and_clean_data_with_mydate('Factors/Corrected_FF_Momentum_Factor.csv', mydate_converter)
df_bond = load_and_clean_data_with_mydate('Factors/DBAA_Monthly_Averages.csv', mydate_converter)
df_credit = load_and_clean_data_with_mydate('Factors/DGS10_Monthly_Averages.csv', mydate_converter, drop_na_columns=['year'])
print(df_credit.head())

   year  month     DGS10  mydate
0  1993      1  6.600000     396
1  1993      2  6.258947     397
2  1993      3  5.975217     398
3  1993      4  5.969524     399
4  1993      5  6.035500     400


In [59]:
# Function to perform AR1 adjustment
def ar1_adjustment(df, period_name):
    df = df.copy()
    df['ret_star'] = df['ret']
    unique_ids = df['id'].unique()
    for unique_id in unique_ids:
        subset = df[df['id'] == unique_id]
        if len(subset) > 1:
            subset = subset.sort_values(by='mydate')
            X = sm.add_constant(subset['ret'].shift(1).dropna())
            y = subset['ret'].iloc[1:]
            if len(X) == len(y):  # Ensure X and y have the same length
                try:
                    model = sm.OLS(y, X).fit()
                    rho = model.params.iloc[1] if len(model.params) > 1 else 0  # Default to 0 if model fitting fails
                    df.loc[df['id'] == unique_id, 'ret_star'] = (df['ret'] - rho * df['ret'].shift(1)) / (1 - rho)
                except Exception as e:
                    print(f"Model fitting failed for id {unique_id} with error: {e}")
    df.to_csv(f'tass4_{period_name}.csv', index=False)
    return df

# Load merged data

merged_df['mydate'] = merged_df['mydate'].astype(int)

# Define periods
pre_crisis_period = merged_df[(merged_df['mydate'] >= 0) & (merged_df['mydate'] < 575)]
crisis_period = merged_df[(merged_df['mydate'] >= 575) & (merged_df['mydate'] <= 593)]
post_crisis_period = merged_df[(merged_df['mydate'] > 593)]

# Perform AR1 adjustment
tass4_pre = ar1_adjustment(pre_crisis_period, 'pre')
tass4_crisis = ar1_adjustment(crisis_period, 'crisis')
tass4_post = ar1_adjustment(post_crisis_period, 'post')


In [60]:
# Function to merge factors with TASS data
def merge_factors(df_tass, factors_list):
    for factor_df in factors_list:
        df_tass = pd.merge(df_tass, factor_df, on=['year', 'month'], how='left', indicator=True, suffixes=('','_remove'))
        df_tass.drop([i for i in df_tass.columns if 'remove' in i], axis=1, inplace=True)
        df_tass = df_tass[df_tass['_merge'] == 'both'].drop('_merge', axis=1)
    return df_tass

# Load the adjusted TASS data for each period
df_tass4_pre = pd.read_csv('tass4_pre.csv')
df_tass4_crisis = pd.read_csv('tass4_crisis.csv')
df_tass4_post = pd.read_csv('tass4_post.csv')

# List of factor dataframes
factors_list = [df_ff, df_fung_hsieh, df_mom, df_bond, df_credit]

# Merging factors with TASS data for each period
df_tass4_pre = merge_factors(df_tass4_pre, factors_list)
df_tass4_crisis = merge_factors(df_tass4_crisis, factors_list)
df_tass4_post = merge_factors(df_tass4_post, factors_list)

# Save the merged dataframes to CSV for further use
df_tass4_pre.to_csv('tass4_pre_merged.csv', index=False)
df_tass4_crisis.to_csv('tass4_crisis_merged.csv', index=False)
df_tass4_post.to_csv('tass4_post_merged.csv', index=False)


In [61]:
def asset_pricing(df, period_name):
    df['lhs'] = df['ret_star'] - df['RF']
    df['excess_ret'] = 0
    df['beta1'] = 0.0
    df['beta2'] = 0.0
    df['beta3'] = 0.0
    df['beta4'] = 0.0
    df['beta5'] = 0.0
    df['beta6'] = 0.0
    df['beta7'] = 0.0
    df['alpha'] = 0.0
    df['stdv'] = 0.0
    df['r2'] = 0.0

    df.rename(columns={'mktrf': 'eq_prem'}, inplace=True)
    unique_ids = df['id'].unique()

    for unique_id in unique_ids:
        sub_df = df[df['id'] == unique_id]
        if len(sub_df) > 1:
            model = sm.OLS(sub_df['lhs'], sm.add_constant(sub_df[['Mkt-RF', 'SMB', 'PTFSBD', 'PTFSFX', 'PTFSCOM', 'year', 'DBAA']])).fit()
            df.loc[df['id'] == unique_id, 'r2'] = model.rsquared
            predictions = model.predict(sm.add_constant(sub_df[['Mkt-RF', 'SMB', 'PTFSBD', 'PTFSFX', 'PTFSCOM', 'year', 'DBAA']]))
            if len(model.params) > 1: df.loc[df['id'] == unique_id, 'beta1'] = model.params.iloc[1]
            if len(model.params) > 2: df.loc[df['id'] == unique_id, 'beta2'] = model.params.iloc[2]
            if len(model.params) > 3: df.loc[df['id'] == unique_id, 'beta3'] = model.params.iloc[3]
            if len(model.params) > 4: df.loc[df['id'] == unique_id, 'beta4'] = model.params.iloc[4]
            if len(model.params) > 5: df.loc[df['id'] == unique_id, 'beta5'] = model.params.iloc[5]
            if len(model.params) > 6: df.loc[df['id'] == unique_id, 'beta6'] = model.params.iloc[6]
            if len(model.params) > 7: df.loc[df['id'] == unique_id, 'beta7'] = model.params.iloc[7]
            if len(model.params) > 0: df.loc[df['id'] == unique_id, 'alpha'] = model.params.iloc[0]
            df.loc[df['id'] == unique_id, 'excess_ret'] = df['lhs'] - predictions + df['alpha']
            df.loc[df['id'] == unique_id, 'stdv'] = df['excess_ret'].std()

    df['excess_ret'] = df['excess_ret'].where(df['ret'].notna())
    df['excess_ret'] = df['excess_ret'].where(df['alpha'].notna())
    df.to_csv(f'tass5_{period_name}.csv', index=False)
    return df

# Perform asset pricing analysis for each period
tass5_pre = asset_pricing(df_tass4_pre, 'pre')
tass5_crisis = asset_pricing(df_tass4_crisis, 'crisis')
tass5_post = asset_pricing(df_tass4_post, 'post')

# Combine all periods into a single dataset
tass5 = pd.concat([tass5_pre, tass5_crisis, tass5_post])
tass5.to_csv('tass5.csv', index=False)


/var/folders/jy/cfb14z9n637_y0c4jp4g2j1h0000gn/T/ipykernel_29273/118254282.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.70340314 10.69702528 10.72526728 10.69119425 10.68186826 10.7402783
 10.70420218 10.71940932 10.67680264 10.71305962 10.69475849 10.72564189
 10.71952427 10.71163142 10.72665276 10.74352887 10.68404206 10.72918996
 10.72366948 10.73380601 10.7222289  10.71132387 10.72392787 10.7078442 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['id'] == unique_id, 'excess_ret'] = df['lhs'] - predictions + df['alpha']


In [62]:
# Preliminarily define "closed" funds during the crisis
tass5 = pd.concat([tass5_pre, tass5_crisis, tass5_post])

# Define the crisis and post periods
tass5['crisis'] = 0
tass5.loc[(tass5['mydate'] >= 573) & (tass5['mydate'] <= 594), 'crisis'] = 1

tass5['post'] = 0
tass5.loc[tass5['mydate'] > 594, 'post'] = 1

# Identify the max and min mydate for each fund
tass5['max_mydate'] = tass5.groupby('id')['mydate'].transform('max')
tass5['min_mydate'] = tass5.groupby('id')['mydate'].transform('min')

# Define funds that closed during the crisis
tass5['closedxcrisis'] = 0
tass5.loc[(tass5['max_mydate'] >= 573) & (tass5['max_mydate'] <= 594) & (tass5['mydate'] == tass5['max_mydate']), 'closedxcrisis'] = 1

# Define firms that closed at least one fund during the crisis
tass5['firm_closedxcrisis'] = tass5.groupby('companyid')['closedxcrisis'].transform('max')

# Save the potential treatment dataset
tass5.to_csv('potential_treat0.csv', index=False)

# Create a dataset of firms that closed at least one fund during the crisis
potential_divcorr = tass5[tass5['firm_closedxcrisis'] == 1].drop(columns=['firm_closedxcrisis', 'closedxcrisis'])
potential_divcorr.to_csv('potential_divcorr.csv', index=False)


In [63]:
# Pre-define treatment
diag1 = tass5.groupby('companyid').agg({'firm_closedxcrisis': 'max', 'mydate': 'max'}).reset_index()
diag1.rename(columns={'firm_closedxcrisis': 'treated'}, inplace=True)
diag1['firm_closed'] = 0
diag1.loc[(diag1['mydate'] >= 573) & (diag1['mydate'] <= 593), 'firm_closed'] = 1
diag1.to_csv('diag1.csv', index=False)

diag2 = tass5.merge(diag1, on='companyid', how='left')
diag2['mydate'] = diag2['mydate_x']
diag2['mydate2'] = diag2['mydate']
diag2 = diag2.groupby('id').agg({'firm_closed': 'max', 'closedxcrisis': 'max', 'treated': 'max', 'mydate': 'max', 'mydate2': 'min'}).reset_index()
diag2['pre_treat'] = 0
diag2.loc[(diag2['firm_closed'] == 0) & (diag2['treated'] == 1) & (diag2['closedxcrisis'] == 0) & (diag2['mydate2'] >= 562) & (diag2['mydate'] >= 605), 'pre_treat'] = 1
diag2 = diag2[['id', 'pre_treat']]
diag2.to_csv('diag2.csv', index=False)

potential_treat0 = pd.read_csv('potential_treat0.csv')
diag2 = pd.read_csv('diag2.csv')

potential_treat0 = potential_treat0.merge(diag2, on='id', how='left').fillna(0)
potential_treat0['pre_treat'] = potential_treat0['pre_treat'].astype(int)
diag3 = potential_treat0.groupby('companyid').agg({'pre_treat': 'max'}).reset_index()
diag3.to_csv('diag3.csv', index=False)

# Clean up intermediate files
import os

files_to_delete = [
    'dataff_fffactors.csv', 'dataff_fung_hsieh.csv', 'dataff_mom.csv', 
    'dataff_bond.csv', 'dataff_credit.csv', 'tass4_pre.csv', 
    'tass4_crisis.csv', 'tass4_post.csv', 'tass5_pre.csv', 
    'tass5_crisis.csv', 'tass5_post.csv'
]

for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)


CAR36

In [64]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('tass5.csv')

# Ensure the dataframe is sorted by 'id' and 'mydate'
df = df.sort_values(by=['id', 'mydate'])

# Calculate the moving average of excess returns over the past 12 months
df['moveave_ex'] = df.groupby('id')['excess_ret'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

# Calculate the sum of squared deviations from the moving average
def sum_sq_ex(group):
    return ((group['excess_ret'] - group['moveave_ex'])**2).rolling(window=12, min_periods=1).sum()

df['sum_sq_ex'] = df.groupby('id').apply(sum_sq_ex).reset_index(level=0, drop=True)

# Calculate the variance of the moving average and the standard deviation
df['var_ex_move'] = df['sum_sq_ex'] / 12
df['stdv_ex_move'] = np.sqrt(df['var_ex_move'])

# Drop unnecessary columns
df = df.drop(columns=['sum_sq_ex', 'var_ex_move'])

# Generate a counter for each id
df['counter'] = df.groupby('id').cumcount() + 1

# Generate the retdum column
df['retdum'] = np.where(df['excess_ret'].notna(), 1, 0)

# Calculate retcounter for various lengths up to 36
for i in range(1, 37):
    df[f'retcounter{i}'] = df.groupby('id')['retdum'].transform(lambda x: x.shift(i).rolling(window=i, min_periods=1).sum())

# Determine the maximum retcounter value for each row
df['retcounter'] = df[[f'retcounter{i}' for i in range(1, 37)]].bfill(axis=1).iloc[:, 0]

# Calculate the CAR values for various lengths up to 36
for i in range(1, 37):
    df[f'CAR{i}'] = df.groupby('id')['excess_ret'].transform(lambda x: x.shift(i).rolling(window=i, min_periods=1).sum())

# Determine the appropriate CAR value based on retcounter
df['CARstar'] = np.nan
for i in range(1, 37):
    df['CARstar'] = np.where(df['retcounter'] >= i, df[f'CAR{i}'], df['CARstar'])

# Adjust retcounter values greater than 36 to 36
df['retcounter'] = np.where(df['retcounter'] > 36, 36, df['retcounter'])

# Calculate the average CAR over the period
df['avgCAR36'] = df['CARstar'] / df['retcounter']

# Save the intermediate result to a CSV file
df.to_csv('car36a.csv', index=False)

# Filter out rows where excess is missing
df = df[df['excess_ret'].notna()]

# Keep only necessary columns
df = df[['id', 'mydate', 'avgCAR36', 'stdv_ex_move']]

# Save the final result to a CSV file
df.to_csv('car36.csv', index=False)


/var/folders/jy/cfb14z9n637_y0c4jp4g2j1h0000gn/T/ipykernel_29273/1806176948.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['sum_sq_ex'] = df.groupby('id').apply(sum_sq_ex).reset_index(level=0, drop=True)


ALL_CLOSURES

In [65]:
import pandas as pd
import numpy as np

# Load the mydate converter
mydate_converter = pd.read_csv('mydate_converter.csv', header=None, names=['year', 'month', 'mydate'], skiprows=[0])

# Load the tass5 dataset
tass5 = pd.read_csv('tass5.csv')

# Calculate min and max mydate for each fund
tass5['min_mydate'] = tass5.groupby('id')['mydate'].transform('min')
tass5['max_mydate'] = tass5.groupby('id')['mydate'].transform('max')

# Generate the starts data
starts = tass5[['id', 'min_mydate']].drop_duplicates()
starts = starts.groupby('min_mydate').size().reset_index(name='starts')
starts.rename(columns={'min_mydate': 'mydate'}, inplace=True)

# Generate the closures data
closures = tass5[['id', 'max_mydate']].drop_duplicates()
closures = closures.groupby('max_mydate').size().reset_index(name='closures')
closures.rename(columns={'max_mydate': 'mydate'}, inplace=True)

# Merge starts and closures data
open_close = pd.merge(starts, closures, on='mydate', how='outer').fillna(0)

# Merge with mydate converter
open_close = pd.merge(open_close, mydate_converter, on='mydate', how='left')

# Save the open_close data to Excel
open_close.to_excel('open_close_TASS2015.xlsx', index=False)

# Aggregate data by year
open_close_year = open_close.groupby('year').sum().reset_index()
open_close_year = open_close_year[['year', 'starts', 'closures']]

# Save the open_close_year data to Excel
open_close_year.to_excel('open_close_TASS2015_year.xlsx', index=False)

# Generate the all_close_treat data
tass5 = tass5[['id', 'companyid', 'mydate', 'min_mydate', 'max_mydate']].drop_duplicates()
tass5['closed'] = np.where((tass5['mydate'] == tass5['max_mydate']) & (tass5['max_mydate'] <= 653), 1, 0)
tass5['opened'] = np.where((tass5['mydate'] == tass5['min_mydate']) & (tass5['min_mydate'] >= 409), 1, 0)

# Collapse data by companyid and mydate
collapsed_data = tass5.groupby(['companyid', 'mydate']).sum().reset_index()

# Create lagged closed variables
for i in range(1, 31):
    collapsed_data[f'L{i}closed'] = collapsed_data.groupby('companyid')['closed'].shift(i).fillna(0)

# Calculate net openings and closings
collapsed_data['net'] = collapsed_data['opened'] - collapsed_data[[f'L{i}closed' for i in range(1, 13)]].sum(axis=1)

# Create open_close and all_close_treat variables
collapsed_data['open_close'] = np.where((collapsed_data['opened'] == 1) & (collapsed_data['net'] <= 0), 1, 0)
collapsed_data['all_close_treat'] = collapsed_data[[f'L{i}closed' for i in range(1, 31)]].max(axis=1)
collapsed_data['time_treat_all'] = np.argmax(collapsed_data[[f'L{i}closed' for i in range(1, 31)]].values >= 1, axis=1) + 1
collapsed_data['time_treat_all'] = np.where(collapsed_data['all_close_treat'] >= 1, collapsed_data['time_treat_all'], 0)

# Keep necessary columns and sort
collapsed_data = collapsed_data[['companyid', 'mydate', 'all_close_treat', 'time_treat_all']].sort_values(by=['companyid', 'mydate'])

# Save the all_close_treat data
collapsed_data.to_csv('all_close_treat.csv', index=False)


RET_CORR_PAIRS

In [66]:
import pandas as pd

# Load the potential_divcorr dataset
potential_divcorr = pd.read_csv('potential_divcorr.csv')

# Step 1: Count the number of funds per firm at any given point in time (simulcounter)
potential_divcorr['simulcounter'] = potential_divcorr.groupby(['companyid', 'mydate']).cumcount() + 1
potential_divcorr['maxsimulcounter'] = potential_divcorr.groupby('companyid')['simulcounter'].transform('max')
simulcounter_df = potential_divcorr.drop_duplicates(subset=['companyid']).copy()
simulcounter_df = simulcounter_df[['companyid', 'maxsimulcounter']].rename(columns={'maxsimulcounter': 'simulcounter'})
# Save simulcounter data
simulcounter_df.to_csv('simulcounter.csv', index=False)
# Step 2: Count the number of funds per firm and filter firms with at least two funds simultaneously
potential_divcorr = pd.merge(potential_divcorr, simulcounter_df, on='companyid', how='inner')
potential_divcorr['simulcounter'] = potential_divcorr['simulcounter_y']
potential_divcorr = potential_divcorr[potential_divcorr['simulcounter'] > 1]

# Step 3: Create a sequential fund identifier within the firm (fundcounter)
fundcounter_df = potential_divcorr.groupby('id').agg({'companyid': 'max', 'mydate': 'min'}).reset_index()
fundcounter_df = fundcounter_df.sort_values(by=['companyid', 'mydate', 'id'])
fundcounter_df['fund_counter'] = fundcounter_df.groupby('companyid').cumcount() + 1
fundcounter_df = fundcounter_df[['companyid', 'id', 'fund_counter']]

# Save fundcounter data
fundcounter_df.to_csv('fundcounter.csv', index=False)

# Step 4: Create a database of error terms from the 7-factor regressions
potential_divcorr['epsilon'] = potential_divcorr['excess_ret'] - potential_divcorr['alpha']
epsilon_df = potential_divcorr[['id', 'mydate', 'epsilon']]

# Save epsilon data
epsilon_df.to_csv('epsilon.csv', index=False)
potential_divcorr.to_csv('potential_divcorr.csv', index=False)

RET_CORR_BIG

In [67]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import glob
import logging

# Initialize the logging functionality
logging.basicConfig(filename='ret_corr_big.log', level=logging.INFO)

# Load datasets
potential_divcorr = pd.read_csv('potential_divcorr.csv')
epsilon = pd.read_csv('epsilon.csv')
simulcounter = pd.read_csv('simulcounter.csv')
fundcounter = pd.read_csv('fundcounter.csv')

# Filter the data and merge necessary information
potential_divcorr = potential_divcorr[['companyid', 'mydate', 'id', 'ret', 'excess_ret', 'alpha']]
potential_divcorr = potential_divcorr.sort_values(by=['id', 'mydate'])
potential_divcorr = potential_divcorr.merge(epsilon, on=['id', 'mydate'], how='left')
potential_divcorr = potential_divcorr.merge(simulcounter, on='companyid', how='inner')

# Filter firms with at least two funds simultaneously
potential_divcorr = potential_divcorr[potential_divcorr['simulcounter'] > 1]

# Pivot data to wide format using actual IDs
df_wide = potential_divcorr.pivot_table(index=["companyid", "mydate"], columns="id", values=["ret", "epsilon"], aggfunc='first')

# Flatten multi-index columns
df_wide.columns = [f"{col[0]}_{col[1]}" for col in df_wide.columns]
df_wide.reset_index(inplace=True)

# Initialize the parameter `last`
last = 65

# Function to calculate and save correlations using actual IDs
def calculate_and_save_correlations(df_wide, last):
    results = []
    columns = df_wide.columns
    fund_ids = [col.split("_")[1] for col in columns if "ret_" in col]
    unique_fund_ids = sorted(set(fund_ids))
    
    for idx1 in range(len(unique_fund_ids)):
        fund_id1 = unique_fund_ids[idx1]
        for idx2 in range(idx1 + 1, len(unique_fund_ids)):
            if idx2 - idx1 >= last:
                break
            fund_id2 = unique_fund_ids[idx2]
            relevant_columns = [f"ret_{fund_id1}", f"ret_{fund_id2}", f"epsilon_{fund_id1}", f"epsilon_{fund_id2}"]
            if all(col in df_wide.columns for col in relevant_columns):
                temp_df = df_wide[['companyid', 'mydate'] + relevant_columns].dropna()
                if len(temp_df) >= 12:
                    correlations = temp_df.corr()
                    ret_corr = correlations.loc[f"ret_{fund_id1}", f"ret_{fund_id2}"]
                    epsilon_corr = correlations.loc[f"epsilon_{fund_id1}", f"epsilon_{fund_id2}"]
                    results.append({
                        "companyid": temp_df['companyid'].iloc[0],
                        "div_cons1": fund_id1,
                        "div_cons2": fund_id2,
                        "div_corr": ret_corr,
                        "corr_eps": epsilon_corr
                    })
                    # Save results to CSV, one file per pair
                    result_df = pd.DataFrame([results[-1]])
                    result_df.to_csv(f"div_corr_{fund_id1}_{fund_id2}.csv", index=False)

# Calculate and save correlations
calculate_and_save_correlations(df_wide, last)

# Combine all correlation files into a single DataFrame
complete_divcorr = pd.DataFrame()
correlation_files = glob.glob("div_corr_*.csv")
for file in correlation_files:
    temp_df = pd.read_csv(file)
    complete_divcorr = pd.concat([complete_divcorr, temp_df], ignore_index=True)
    complete_divcorr.sort_values(by=['companyid', 'div_cons1', 'div_cons2'], inplace=True)
    complete_divcorr['div_corr_id'] = range(1, len(complete_divcorr)+1)
complete_divcorr.to_csv("complete_divcorr.csv", index=False)


In [68]:
for x in range(1, 100000):
    n = x + 1
    try:
        os.remove(glob('div_corr_*.csv'))
    except FileNotFoundError:
        continue

TypeError: 'module' object is not callable

RET_CORR_INTEGRATE -> zit al bij BIG 

Enig hier is dat ipv tm 65, de laatste 65 worden gepakt, nog niet helemaal duidelijk wat het uitmaakt

RET_CORR_INTEGRATE 2

In [69]:
import pandas as pd
import numpy as np

# Load necessary datasets
complete_divcorr = pd.read_csv('complete_divcorr.csv')
potential_treat0 = pd.read_csv('potential_treat0.csv')

# Step 1: Make a list of every fund (id) in complete_divcorr and call it potential_treat1
divcorr_id1 = complete_divcorr[['div_cons1']].rename(columns={'div_cons1': 'id'})
divcorr_id2 = complete_divcorr[['div_cons2']].rename(columns={'div_cons2': 'id'})
potential_treat1 = pd.concat([divcorr_id1, divcorr_id2]).drop_duplicates().sort_values(by='id')
potential_treat1.to_csv('potential_treat1.csv', index=False)

# Step 2: Merge in everything from potential_treat0
potential_treat2 = potential_treat0.merge(potential_treat1, on='id', how='left')
potential_treat2['closedxcrisis'].fillna(0, inplace=True)
potential_treat2['closedxcrisis'] = potential_treat2['closedxcrisis'].astype(int)
potential_treat2['fund_closedxcrisis'] = potential_treat2.groupby('id')['closedxcrisis'].transform('max')
potential_treat2.to_csv('potential_treat2.csv', index=False)

# Step 3: Create a file of ids with min_mydate
minmydate = potential_treat2[potential_treat2['mydate'] == potential_treat2['min_mydate']]
minmydate = minmydate[['id', 'min_mydate']]
minmydate.to_csv('minmydate.csv', index=False)

oth_minmydate = minmydate.rename(columns={'id': 'other_fund', 'min_mydate': 'oth_minmydate'})
oth_minmydate.to_csv('oth_minmydate.csv', index=False)

# Step 4: Eliminate "twins"
# Batch 1
batch1 = complete_divcorr[['companyid', 'div_cons1', 'div_cons2', 'div_corr', 'corr_eps']]
batch1 = batch1.rename(columns={'div_cons1': 'id', 'div_cons2': 'other_fund'})
batch1.to_csv('batch1.csv', index=False)

# Batch 2
batch2 = complete_divcorr[['companyid', 'div_cons1', 'div_cons2', 'div_corr', 'corr_eps']]
batch2 = batch2.rename(columns={'div_cons2': 'id', 'div_cons1': 'other_fund'})
batch2 = pd.concat([batch2, batch1]).drop_duplicates().sort_values(by=['companyid', 'id', 'other_fund'])

batch2 = batch2.merge(minmydate, on='id', how='inner').rename(columns={'min_mydate': 'id_minmydate'})
batch2 = batch2.merge(oth_minmydate, on='other_fund', how='inner')
batch2.to_csv('batch2.csv', index=False)

# Batch 3: Focus on (unique) funds with div_corr > 0.985
batch3 = batch2.copy()
batch3['corr_eps'].replace(-999, np.nan, inplace=True)
batch3 = batch3[batch3['div_corr'] > 0.985]
batch3.drop_duplicates(subset=['id', 'other_fund'], inplace=True)

batch3['keep_id'] = np.where(batch3['id_minmydate'] <= batch3['oth_minmydate'], 1, 0)
batch3['keep_other_fund'] = np.where(batch3['oth_minmydate'] < batch3['id_minmydate'], 1, 0)
batch3.to_csv('batch3.csv', index=False)

# Batch 4: Identify other_fund ids when keep_id==0 for an id and rename other_fund id
batch4 = batch3[batch3['keep_id'] == 0][['other_fund', 'div_corr']]
batch4 = batch4.rename(columns={'other_fund': 'id'})
batch4.to_csv('batch4.csv', index=False)

# Highcorr_set: Set of funds to be kept, even though they are highly correlated
highcorr_set = pd.concat([
    batch3[batch3['keep_other_fund'] == 0][['id', 'div_corr']],
    batch4
]).drop_duplicates(subset=['id']).sort_values(by='id')
highcorr_set.to_csv('highcorr_set.csv', index=False)

# Dropcorr_set: Set of "twin" funds to be dropped
dropcorr_set = batch3[['id']].drop_duplicates().sort_values(by='id')
dropcorr_set = dropcorr_set.merge(highcorr_set, on='id', how='left', indicator=True)
dropcorr_set = dropcorr_set[dropcorr_set['_merge'] == 'left_only'].drop(columns=['_merge'])
dropcorr_set.to_csv('dropcorr_set.csv', index=False)

# Define the test set
intheset = potential_treat2.copy()
intheset = intheset.merge(dropcorr_set, on='id', how='left', indicator=True)
intheset['intheset'] = 1
intheset.loc[intheset['crisis'] == 1, 'intheset'] = 0
intheset.loc[intheset['fund_closedxcrisis'] == 1, 'intheset'] = 0
intheset.loc[intheset['min_mydate'] > 562, 'intheset'] = 0
intheset.loc[intheset['_merge'] == 'both', 'intheset'] = 0
intheset.drop(columns=['_merge'], inplace=True)
intheset = intheset[intheset['intheset'] == 1]

intheset['intheset2'] = np.where((intheset['mydate'] <= 625) & (intheset['mydate'] >= 617), 0, intheset['intheset'])
intheset['counter'] = intheset.groupby('id').cumcount() + 1
intheset['maxcounter'] = intheset.groupby('id')['counter'].transform('max')
intheset['intheset'] = np.where(intheset['maxcounter'] < 12, 0, intheset['intheset'])

intheset = intheset[intheset['intheset'] == 1][['id', 'mydate', 'intheset', 'intheset2']]
intheset.to_csv('intheset.csv', index=False)

# Bring in all pairwise correlations
integrate2 = complete_divcorr[['companyid', 'div_cons1', 'div_cons2', 'div_corr', 'corr_eps']]
integrate2 = integrate2.rename(columns={'div_cons2': 'id', 'div_cons1': 'other_fund'})
integrate2 = pd.concat([integrate2, batch1]).drop_duplicates().sort_values(by=['companyid', 'id', 'other_fund'])
integrate2['corr_eps'].replace(-999, np.nan, inplace=True)
integrate2.to_csv('integrate2.csv', index=False)

# Identify funds with div_corr that were closed during the crisis
closed = potential_treat2[potential_treat2['closedxcrisis'] == 1][['id']]
closed.to_csv('closed.csv', index=False)

# Merge closed fund with list of funds w/div_corr to find the "real" treatments
integrate3 = closed.merge(integrate2, on='id', how='inner')
integrate3['treatment99'] = np.where(integrate3['div_corr'] < 0.985, 1, 0)
integrate3['treatment90'] = np.where(integrate3['div_corr'] < 0.895, 1, 0)
integrate3['treatment99_eps'] = np.where((integrate3['corr_eps'] < 0.985) & (integrate3['corr_eps'].notna()), 1, 0)
integrate3['treatment90_eps'] = np.where((integrate3['corr_eps'] < 0.895) & (integrate3['corr_eps'].notna()), 1, 0)

integrate3 = integrate3.rename(columns={'id': 'treated_id', 'other_fund': 'id'}).sort_values(by=['companyid', 'id'])
integrate3.to_csv('integrate3.csv', index=False)

# Merge with potential_treat2
integrate3 = integrate3.merge(potential_treat2, on=['companyid', 'id'], how='outer')

# DIV_CORR < 0.985
integrate3['closedxcrisis2'] = integrate3['closedxcrisis']
integrate3['closedxcrisis3'] = integrate3['closedxcrisis']
integrate3['closedxcrisis4'] = integrate3['closedxcrisis']
integrate3.loc[integrate3['div_corr'] >= 0.985, 'closedxcrisis'] = 0
integrate3['fund_closedxcrisis'] = integrate3.groupby('id')['closedxcrisis'].transform('max')



/var/folders/jy/cfb14z9n637_y0c4jp4g2j1h0000gn/T/ipykernel_29273/3981866167.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  potential_treat2['closedxcrisis'].fillna(0, inplace=True)
/var/folders/jy/cfb14z9n637_y0c4jp4g2j1h0000gn/T/ipykernel_29273/3981866167.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

In [70]:


# Instantaneous and constant FIRM measures of closure during the crisis
integrate3['firm_closedxcrisis'] = integrate3.groupby(['companyid', 'mydate'])['closedxcrisis'].transform('max')
integrate3['max_firm_closedxcrisis'] = integrate3.groupby('companyid')['closedxcrisis'].transform('max')

# A fund cannot be treated if it started within one year of the financial crisis
integrate3.loc[integrate3['min_mydate'] > 562, 'max_firm_closedxcrisis'] = 0
integrate3.loc[integrate3['div_corr'] >= 0.985, 'max_firm_closedxcrisis'] = 0

# crisis_treat24 equals one for two years after the end of the crisis
integrate3['crisis_treat24'] = 0
integrate3.loc[(integrate3['max_firm_closedxcrisis'] == 1) & (integrate3['mydate'] >= 595) & (integrate3['mydate'] <= 619), 'crisis_treat24'] = 1

# DIV_CORR < 0.895
integrate3.loc[integrate3['div_corr'] >= 0.895, 'closedxcrisis2'] = 0
integrate3['fund_closedxcrisis2'] = integrate3.groupby('id')['closedxcrisis2'].transform('max')

# Instantaneous and constant FIRM measures of closure during the crisis
integrate3['firm_closedxcrisis2'] = integrate3.groupby(['companyid', 'mydate'])['closedxcrisis2'].transform('max')
integrate3['max_firm_closedxcrisis2'] = integrate3.groupby('companyid')['closedxcrisis2'].transform('max')

# A fund cannot be treated if it started within one year of the financial crisis
integrate3.loc[integrate3['min_mydate'] > 562, 'max_firm_closedxcrisis2'] = 0
integrate3.loc[integrate3['div_corr'] >= 0.895, 'max_firm_closedxcrisis2'] = 0

# crisis_treat24 equals one for two years after the end of the crisis
integrate3['crisis_treat24_90'] = 0
integrate3.loc[(integrate3['max_firm_closedxcrisis2'] == 1) & (integrate3['mydate'] >= 595) & (integrate3['mydate'] <= 619), 'crisis_treat24_90'] = 1

# CORR_EPS < 0.985
integrate3.loc[integrate3['corr_eps'] >= 0.985, 'closedxcrisis3'] = 0
integrate3['fund_closedxcrisis3'] = integrate3.groupby('id')['closedxcrisis3'].transform('max')

# Instantaneous and constant FIRM measures of closure during the crisis
integrate3['firm_closedxcrisis3'] = integrate3.groupby(['companyid', 'mydate'])['closedxcrisis3'].transform('max')
integrate3['max_firm_closedxcrisis3'] = integrate3.groupby('companyid')['closedxcrisis3'].transform('max')

# A fund cannot be treated if it started within one year of the financial crisis
integrate3.loc[integrate3['min_mydate'] > 562, 'max_firm_closedxcrisis3'] = 0
integrate3.loc[integrate3['div_corr'] >= 0.985, 'max_firm_closedxcrisis3'] = 0

# crisis_treat24 equals one for two years after the end of the crisis
integrate3['crisis_treat24_99eps'] = 0
integrate3.loc[(integrate3['max_firm_closedxcrisis3'] == 1) & (integrate3['mydate'] >= 595) & (integrate3['mydate'] <= 619), 'crisis_treat24_99eps'] = 1

# CORR_EPS < 0.895
integrate3.loc[integrate3['corr_eps'] >= 0.895, 'closedxcrisis4'] = 0
integrate3['fund_closedxcrisis4'] = integrate3.groupby('id')['closedxcrisis4'].transform('max')

# Instantaneous and constant FIRM measures of closure during the crisis
integrate3['firm_closedxcrisis4'] = integrate3.groupby(['companyid', 'mydate'])['closedxcrisis4'].transform('max')
integrate3['max_firm_closedxcrisis4'] = integrate3.groupby('companyid')['closedxcrisis4'].transform('max')

# A fund cannot be treated if it started within one year of the financial crisis
integrate3.loc[integrate3['min_mydate'] > 562, 'max_firm_closedxcrisis4'] = 0
integrate3.loc[integrate3['div_corr'] >= 0.895, 'max_firm_closedxcrisis4'] = 0

# crisis_treat24 equals one for two years after the end of the crisis
integrate3['crisis_treat24_90eps'] = 0
integrate3.loc[(integrate3['max_firm_closedxcrisis4'] == 1) & (integrate3['mydate'] >= 595) & (integrate3['mydate'] <= 619), 'crisis_treat24_90eps'] = 1

integrate3.to_csv('integrate4.csv', index=False)

# Clean up
import os
files_to_erase = [
    'batch1.csv', 'batch3.csv', 'batch4.csv', 'potential_treat1.csv', 
    'integrate2.csv', 'highcorr_set.csv'
]

for file in files_to_erase:
    if os.path.exists(file):
        os.remove(file)

print("Integration completed and temporary files cleaned up.")


Integration completed and temporary files cleaned up.
